*Importar librerias*

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import os
from datetime import datetime

In [2]:
#ruta del dataset
data_path = os.path.join(os.getcwd(),'..','data_sintetica','dataset_morosidad.csv')
df = pd.read_csv(data_path)

df.head

<bound method NDFrame.head of        feature_1  feature_2  feature_3  feature_4  feature_5  feature_6  \
0       0.470260   1.659982  -1.571727   1.796075   0.261384  -0.389445   
1       0.074141   1.164003  -1.246876   1.300156   0.337223   2.291755   
2       2.263460   2.062194  -0.941923  -5.102603   0.874995   1.558469   
3      -0.291488   0.622700  -0.546771  -1.429549  -0.130805   2.028093   
4      -2.319668  -0.472974  -0.594128   0.540503  -0.083435  -1.789821   
...          ...        ...        ...        ...        ...        ...   
49995   0.117983   4.161858  -0.188374  -1.180606   0.328965  -4.307407   
49996  -0.195621   2.196535  -1.162599  -4.120151  -0.532278   3.012379   
49997  -2.925590   0.973396  -0.014262   0.814898   0.852992   1.508443   
49998   1.926403  -1.930037  -0.914429  -0.579894  -1.284220  -2.416351   
49999   0.972349   2.572045  -1.306468  -3.826070   0.978887   2.530766   

       feature_7  feature_8  feature_9  feature_10  moroso  
0      -

In [3]:
#Cerar columna ID
if 'ID_cliente' not in df.columns:
    df.insert(0, 'ID_Cliente', [f"C{str(i).zfill(6)}" for i in range(1, len(df)+1)])

df.head


<bound method NDFrame.head of       ID_Cliente  feature_1  feature_2  feature_3  feature_4  feature_5  \
0        C000001   0.470260   1.659982  -1.571727   1.796075   0.261384   
1        C000002   0.074141   1.164003  -1.246876   1.300156   0.337223   
2        C000003   2.263460   2.062194  -0.941923  -5.102603   0.874995   
3        C000004  -0.291488   0.622700  -0.546771  -1.429549  -0.130805   
4        C000005  -2.319668  -0.472974  -0.594128   0.540503  -0.083435   
...          ...        ...        ...        ...        ...        ...   
49995    C049996   0.117983   4.161858  -0.188374  -1.180606   0.328965   
49996    C049997  -0.195621   2.196535  -1.162599  -4.120151  -0.532278   
49997    C049998  -2.925590   0.973396  -0.014262   0.814898   0.852992   
49998    C049999   1.926403  -1.930037  -0.914429  -0.579894  -1.284220   
49999    C050000   0.972349   2.572045  -1.306468  -3.826070   0.978887   

       feature_6  feature_7  feature_8  feature_9  feature_10  moroso

In [4]:
#Monto de credito

df['monto_credito'] = (
    df['feature_1'] * 1000+
    df['feature_2'] * 200+
    np.random.normal(0, 500, size=len(df))
)

df[['feature_1', 'feature_2', 'monto_credito']].head

<bound method NDFrame.head of        feature_1  feature_2  monto_credito
0       0.470260   1.659982     740.636574
1       0.074141   1.164003     337.458806
2       2.263460   2.062194    2731.252641
3      -0.291488   0.622700    -759.574214
4      -2.319668  -0.472974   -1462.689388
...          ...        ...            ...
49995   0.117983   4.161858     881.298225
49996  -0.195621   2.196535    -309.436689
49997  -2.925590   0.973396   -3059.357739
49998   1.926403  -1.930037    2013.860767
49999   0.972349   2.572045    1953.731588

[50000 rows x 3 columns]>

In [5]:
X = df[['feature_1']]
y = df[['monto_credito']]

In [6]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size = 0.3, random_state=42
)

#Ids de los casos de test
id_test = df.loc[y_test.index, 'ID_Cliente'].reset_index(drop=True)

X_train.shape, X_test.shape

((35000, 1), (15000, 1))

In [7]:
#entrenando el modelo de regresion

reg = LinearRegression()
reg.fit(X_train, y_train)

,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


In [8]:
y_pred = reg.predict(X_test)
y_pred[:5]

array([[  764.47178104],
       [  -19.93217084],
       [-1316.5071625 ],
       [  182.79898247],
       [-2070.29595815]])

In [9]:
#Evaluar el modelo
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print ("MSE: ", mse)
print ("R2: ", r2)

MSE:  357373.2483894413
R2:  0.7352537448660967


In [12]:
print(y_pred.shape)

(15000, 1)


In [19]:
print('id_test:', id_test.shape, type(id_test))
print('y_test_reset', y_test_reset.shape, type(y_test_reset))
print('y_pred_series', y_pred_series.shape, type(y_pred_series))

id_test: (15000,) <class 'pandas.core.series.Series'>
y_test_reset (15000,) <class 'pandas.core.series.Series'>
y_pred_series (15000,) <class 'pandas.core.series.Series'>


In [18]:
#predicciones
y_pred = reg.predict(X_test)

#Aplanar el array a ID
y_pred = y_pred.ravel()

#Convertir  y_pred a Series
y_pred_series = pd.Series(y_pred)

#Reiniciar índices de y_test
y_test_reset = y_test.reset_index(drop=True)
y_test_reset = y_test_reset.squeeze()

#Creando DF del resultado
df_resultados = pd.DataFrame({
    'ID_Cliente': id_test,
    'monto_credito_real': y_test_reset,
    'monto_credito_predicho': y_pred_series,
    'error_abosoluto': np.abs(y_test_reset - y_pred_series)
})

df_resultados.head()

,ID_Cliente,monto_credito_real,monto_credito_predicho,error_abosoluto
0,C033554,560.539041,764.471781,203.932740
1,C009428,-443.886951,-19.932171,423.954780
2,C000200,-679.122986,-1316.507162,637.384176
3,C012448,539.805478,182.798982,357.006495
4,C039490,-2798.365496,-2070.295958,728.069538


In [22]:
resultados_dir = os.path.join(os.getcwd(),'..','resultado de modelos')
os.makedirs(resultados_dir, exist_ok=True)

#Nombre del archivo
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_file = f"RegresionLineal_resultados_{timestamp}.csv"
output_path = os.path.join(resultados_dir, output_file)

#Guardar
df_resultados.to_csv(output_path, index=False, encoding='utf-8-sig')

print(f"Archivo de resultados gaurdados en: {output_path}")

Archivo de resultados gaurdados en: d:\IC\Machine_learning_1\semana_1\..\resultado de modelos\RegresionLineal_resultados_20250707_215441.csv
